In [1]:
import re
import nltk
nltk.download('brown')
from nltk.corpus import brown
import itertools

corpus = []

for cat in ['news']:
    for text_id in brown.fileids(cat):
        raw_text = list(itertools.chain.from_iterable(brown.sents(text_id)))
        text = ' '.join(raw_text)
        text = text.lower()
        text.replace('\n', ' ')
        text = re.sub('[^a-z ]+', '', text)
        corpus.append([w for w in text.split() if w != ''])

[nltk_data] Downloading package brown to /Users/fahimehb/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [4]:
from collections import Counter
import random, math

def subsample_frequent_words(corpus):
    filtered_corpus = []
    word_counts = dict(Counter(list(itertools.chain.from_iterable(corpus))))
    sum_word_counts = sum(list(word_counts.values()))
    word_counts = {word: word_counts[word]/float(sum_word_counts) for word in word_counts}
    for text in corpus:
        filtered_corpus.append([])
        for word in text:
            if random.random() < (1+math.sqrt(word_counts[word] * 1e3)) * 1e-3 / float(word_counts[word]):
                filtered_corpus[-1].append(word)
    return filtered_corpus

In [82]:
#corpus = subsample_frequent_words(corpus)
from cell import utils
corpus = utils.read_list_of_lists_from_csv("/Users/fahimehb/Documents/NPP_GNN_project/dat/walks/VISp/test_layer/Undirected_Weighted_node2vec/l_10000_p_1_q_1/", "walk0.csv")

vocabulary = set(itertools.chain.from_iterable(corpus))

word_to_index = {w: idx for (idx, w) in enumerate(vocabulary)}
index_to_word = {idx: w for (idx, w) in enumerate(vocabulary)}

In [85]:
index_to_word

{0: '78',
 1: '84',
 2: '30',
 3: '54',
 4: '0',
 5: '2',
 6: '7',
 7: '73',
 8: '38',
 9: '27',
 10: '88',
 11: '6',
 12: '48',
 13: '8',
 14: '71',
 15: '52',
 16: '65',
 17: '58',
 18: '4',
 19: '67',
 20: '76',
 21: '56',
 22: '68',
 23: '9',
 24: '60',
 25: '20',
 26: '64',
 27: '24',
 28: '79',
 29: '55',
 30: '51',
 31: '49',
 32: '1',
 33: '46',
 34: '41',
 35: '61',
 36: '81',
 37: '25',
 38: '5',
 39: '10',
 40: '37',
 41: '80',
 42: '31',
 43: '28',
 44: '70',
 45: '69',
 46: '57',
 47: '50',
 48: '82',
 49: '43',
 50: '35',
 51: '23',
 52: '19',
 53: '18',
 54: '90',
 55: '36',
 56: '15',
 57: '45',
 58: '89',
 59: '3',
 60: '33',
 61: '62',
 62: '83',
 63: '11',
 64: '63',
 65: '22',
 66: '42',
 67: '77',
 68: '44',
 69: '21',
 70: '29',
 71: '66',
 72: '16',
 73: '47',
 74: '86',
 75: '72',
 76: '92',
 77: '34',
 78: '12',
 79: '74',
 80: '13',
 81: '53',
 82: '85',
 83: '75',
 84: '40',
 85: '17',
 86: '91',
 87: '32',
 88: '14',
 89: '87',
 90: '26',
 91: '39',
 92: '59

In [86]:
len(word_to_index)

93

In [8]:
# import numpy as np

# context_tuple_list = []
# w = 4

# for text in corpus:
#     for i, word in enumerate(text):
#         first_context_word_index = max(0,i-w)
#         last_context_word_index = min(i+w, len(text))
#         for j in range(first_context_word_index, last_context_word_index):
#             if i!=j:
#                 context_tuple_list.append((word, text[j]))
# print("There are {} pairs of target and context words".format(len(context_tuple_list)))

There are 474043 pairs of target and context words


In [87]:
import torch
import torch.nn  as  nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F


class Word2Vec(nn.Module):

    def __init__(self, embedding_size, vocab_size):
        super(Word2Vec, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.linear = nn.Linear(embedding_size, vocab_size)
        
    def forward(self, context_word):
        emb = self.embeddings(context_word)
        hidden = self.linear(emb)
        out = F.log_softmax(hidden)
        return out

In [16]:
class EarlyStopping():
    def __init__(self, patience=5, min_percent_gain=0.1):
        self.patience = patience
        self.loss_list = []
        self.min_percent_gain = min_percent_gain / 100.
        
    def update_loss(self, loss):
        self.loss_list.append(loss)
        if len(self.loss_list) > self.patience:
            del self.loss_list[0]
    
    def stop_training(self):
        if len(self.loss_list) == 1:
            return False
        gain = (max(self.loss_list) - min(self.loss_list)) / max(self.loss_list)
        print("Loss gain: {}%".format(round(100*gain,2)))
        if gain < self.min_percent_gain:
            return True
        else:
            return False

In [89]:
# vocabulary_size = len(vocabulary)

# net = Word2Vec(embedding_size=2, vocab_size=vocabulary_size)
# loss_function = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters())
# early_stopping = EarlyStopping()
# context_tensor_list = []

# for target, context in context_tuple_list:
#     target_tensor = autograd.Variable(torch.LongTensor([word_to_index[target]]))
#     context_tensor = autograd.Variable(torch.LongTensor([word_to_index[context]]))
#     context_tensor_list.append((target_tensor, context_tensor))
    
# while True:
#     losses = []
#     for target_tensor, context_tensor in context_tensor_list:
#         net.zero_grad()
#         log_probs = net(context_tensor)
#         loss = loss_function(log_probs, target_tensor)
#         loss.backward()
#         optimizer.step()
#         losses.append(loss.data)
#         print(loss)
#     print("Loss: ", np.mean(losses))
#     early_stopping.update_loss(np.mean(losses))
#     if early_stopping.stop_training():
#         break

In [19]:
import random

def get_batches(context_tuple_list, batch_size=100):
    random.shuffle(context_tuple_list)
    batches = []
    batch_target, batch_context, batch_negative = [], [], []
    for i in range(len(context_tuple_list)):
        batch_target.append(word_to_index[context_tuple_list[i][0]])
        batch_context.append(word_to_index[context_tuple_list[i][1]])
        batch_negative.append([word_to_index[w] for w in context_tuple_list[i][2]])
        if (i+1) % batch_size == 0 or i == len(context_tuple_list)-1:
            tensor_target = autograd.Variable(torch.from_numpy(np.array(batch_target)).long())
            tensor_context = autograd.Variable(torch.from_numpy(np.array(batch_context)).long())
            tensor_negative = autograd.Variable(torch.from_numpy(np.array(batch_negative)).long())
            batches.append((tensor_target, tensor_context, tensor_negative))
            batch_target, batch_context, batch_negative = [], [], []
    return batches

In [20]:
from numpy.random import multinomial

def sample_negative(sample_size):
    sample_probability = {}
    word_counts = dict(Counter(list(itertools.chain.from_iterable(corpus))))
    normalizing_factor = sum([v**0.75 for v in word_counts.values()])
    for word in word_counts:
        sample_probability[word] = word_counts[word]**0.75 / normalizing_factor
    words = np.array(list(word_counts.keys()))
    while True:
        word_list = []
        sampled_index = np.array(multinomial(sample_size, list(sample_probability.values())))
        for index, count in enumerate(sampled_index):
            for _ in range(count):
                 word_list.append(words[index])
        yield word_list

In [29]:
[len(corpus[i]) for i in range(len(corpus))]

[1518,
 1564,
 1521,
 1563,
 1539,
 1541,
 1546,
 1524,
 1488,
 1541,
 1534,
 1554,
 1531,
 1524,
 1531,
 1610,
 1557,
 1584,
 1527,
 1547,
 1561,
 1570,
 1536,
 1515,
 1527,
 1539,
 1508,
 1540,
 1602,
 1574,
 1550,
 1536,
 1511,
 1524,
 1526,
 1478,
 1555,
 1506,
 1563,
 1585,
 1509,
 1546,
 1567,
 1549]

In [146]:
import numpy as np

context_tuple_list = []
w = 2
negative_samples = sample_negative(1)

k =0 
for text in corpus:
    print(len(text))
    for i, word in enumerate(text):
        first_context_word_index = max(0,i-w)
        last_context_word_index = min(i+w, len(text))
        for j in range(first_context_word_index, last_context_word_index):
            if i!=j:
                context_tuple_list.append((word, text[j], next(negative_samples)))
print("There are {} pairs of target and context words".format(len(context_tuple_list)))

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
There are 2789628 pairs of target and context words


In [91]:
context_tuple_list

[('0', '37', ['8', '13', '55', '89', '23']),
 ('37', '0', ['44', '58', '52', '49', '55']),
 ('37', '43', ['63', '61', '53', '36', '60']),
 ('43', '0', ['1', '78', '8', '58', '90']),
 ('43', '37', ['1', '33', '58', '38', '12']),
 ('43', '46', ['82', '9', '34', '40', '23']),
 ('46', '37', ['43', '82', '74', '39', '12']),
 ('46', '43', ['78', '30', '67', '90', '64']),
 ('46', '75', ['43', '75', '78', '14', '40']),
 ('75', '43', ['45', '74', '8', '29', '52']),
 ('75', '46', ['43', '48', '27', '49', '40']),
 ('75', '45', ['9', '1', '61', '71', '87']),
 ('45', '46', ['80', '8', '58', '86', '64']),
 ('45', '75', ['82', '63', '76', '16', '5']),
 ('45', '32', ['43', '63', '61', '10', '60']),
 ('32', '75', ['16', '42', '25', '55', '35']),
 ('32', '45', ['17', '63', '33', '6', '15']),
 ('32', '92', ['37', '74', '41', '65', '86']),
 ('92', '45', ['44', '26', '47', '39', '23']),
 ('92', '32', ['45', '22', '74', '10', '5']),
 ('92', '0', ['52', '87', '10', '34', '13']),
 ('0', '32', ['74', '87', '56

In [150]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F


class Word2Vec(nn.Module):

    def __init__(self, embedding_size, vocab_size):
        super(Word2Vec, self).__init__()
        self.embeddings_target = nn.Embedding(vocab_size, embedding_size)
        self.embeddings_context = nn.Embedding(vocab_size, embedding_size)

    def forward(self, target_word, context_word, negative_example):
        emb_target = self.embeddings_target(target_word)
        emb_context = self.embeddings_context(context_word)
        emb_product = torch.mul(emb_target, emb_context)
        emb_product = torch.sum(emb_product, dim=1)
        out = torch.sum(F.logsigmoid(emb_product))
        emb_negative = self.embeddings_context(negative_example)
        emb_product = torch.bmm(emb_negative, emb_target.unsqueeze(2))
        emb_product = torch.sum(emb_product, dim=1)
        out += torch.sum(F.logsigmoid(-emb_product))
        return -out

In [151]:
import time

vocabulary_size = len(vocabulary)

loss_function = nn.CrossEntropyLoss()
net = Word2Vec(embedding_size=10, vocab_size=vocabulary_size)
optimizer = optim.Adam(net.parameters())
early_stopping = EarlyStopping(patience=5, min_percent_gain=1)

while True:
    losses = []
    context_tuple_batches = get_batches(context_tuple_list, batch_size=2000)
    for i in range(len(context_tuple_batches)):
        net.zero_grad()
        target_tensor, context_tensor, negative_tensor = context_tuple_batches[i]
        loss = net(target_tensor, context_tensor, negative_tensor)
        loss.backward()
        optimizer.step()
        losses.append(loss.data)
        #print(loss)
    print("Loss: ", np.mean(losses))
    early_stopping.update_loss(np.mean(losses))
    if early_stopping.stop_training():
        break

Loss:  3715.6357
Loss:  2820.4768
Loss gain: 24.09%
Loss:  2772.9517
Loss gain: 25.37%
Loss:  2766.7708
Loss gain: 25.54%
Loss:  2762.6975
Loss gain: 25.65%
Loss:  2761.5132
Loss gain: 2.09%
Loss:  2761.0781
Loss gain: 0.43%


In [152]:
print("Done!")

Done!


In [153]:
import numpy as np

def get_closest_word(word, topn=5):
    word_distance = []
    emb = net.embeddings_target
    pdist = nn.PairwiseDistance()
    i = word_to_index[word]
    lookup_tensor_i = torch.tensor([i], dtype=torch.long)
    v_i = emb(lookup_tensor_i)
    for j in range(len(vocabulary)):
        if j != i:
            lookup_tensor_j = torch.tensor([j], dtype=torch.long)
            v_j = emb(lookup_tensor_j)
            word_distance.append((index_to_word[j], float(pdist(v_i, v_j))))
    word_distance.sort(key=lambda x: x[1])
    return word_distance[:topn]

In [154]:
len(vocabulary)

93

In [155]:
vectors = net.embeddings_target.weight.detach().numpy()

In [156]:
vectors.shape

(93, 10)

In [157]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [164]:
scaled_data = preprocessing.StandardScaler().fit_transform(vectors)
pca = PCA(n_components=10)
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)

per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]

plt.figure(figsize=(20,5))
plt.bar(x=range(1, len(per_var)+1), height=per_var, tick_label= labels)
plt.xticks(rotation = 90)
plt.show()

In [159]:
cldf = utils.read_visp_npp_cldf()
cldf.head()

Reading cldf from: //Users/fahimehb/Documents/NPP_GNN_project/dat/cl_df_VISp_annotation.csv


,dendcluster_id,cluster_label,colors,cluster_color,subclass_id,subclass_label,subclass_color,class_id,class_label,class_color
cluster_id,,,,,,,,,,
0,1,L2/3 IT VISp Rrad,#D9F077,#D9F077,7,L2/3 IT,#94D9A1,2,Glutamatergic,#27AAE1
1,2,L2/3 IT VISp Adamts2,#A6E6A9,#A6E6A9,7,L2/3 IT,#94D9A1,2,Glutamatergic,#27AAE1
2,3,L2/3 IT VISp Agmat,#7AE6AB,#7AE6AB,7,L2/3 IT,#94D9A1,2,Glutamatergic,#27AAE1
3,7,L4 IT VISp Rspo1,#00979D,#00979D,8,L4,#00979D,2,Glutamatergic,#27AAE1
4,8,L5 IT VISp Hsd11b1 Endou,#00DDC5,#00DDC5,9,L5 IT,#008A61,2,Glutamatergic,#27AAE1


In [160]:
from cell import  analysis, plot_utils
from sklearn.decomposition import PCA


In [161]:
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(vectors)
pca_data = analysis.summarize_walk_embedding_results(gensim_dict={"model": principalComponents},
                                                 index = [str(i) for i in range(93)],
                                                 ndim=3, 
                                                 cl_df=cldf)

In [162]:
%matplotlib qt5
plot_utils.plot_embedding(data=pca_data, cl_df=cldf, plot_dim=3, plot_size=(5,5))